# Interdiction de plus courts chemins

MPRO 2018/2018 - RORT

*Guillaume CROGNIER, Guillaume DALLE, Olivier RIGAL*

## 2. Partie heuristique

Installer les packages nécessaires

In [14]:
using Pkg
# Pkg.update()

Importer les packages nécessaires

In [15]:
using Random
using LightGraphs, SimpleWeightedGraphs

┌ Info: Precompiling SimpleWeightedGraphs [47aef6b3-ad0c-573a-a1e2-d07658019622]
└ @ Base loading.jl:1192


Définir une graine aléatoire pour des expériences reproductibles

In [16]:
Random.seed!(63);

Importer le générateur d'instances

In [17]:
include("instances_io.jl");

Génération de l'instance

In [18]:
l = 5 # number of lines in the grid-like graph
c = 5 # number of columns in the grid-like graph
k = 10 # number of edges the adversary can penalize
maxc = 5 # range of initial edge cost
maxd = 50 # range of penalized edge cost
sv = generate(l, c, k, maxc, maxd);

Liste des arcs

In [48]:
function get_arcs(sv::Data)
    arcs::Array{Tuple{Int64, Int64}, 1} = []
    for u in 1:sv.n, v in 1:sv.n
        if sv.adj[u][v]
            push!(arcs, (u, v))
        end
    end
    return arcs
end

get_arcs (generic function with 1 method)

In [58]:
using LightGraphs, SimpleWeightedGraphs

function shortest_path(sv::Data, x::Dict{Tuple{Int64,Int64}, Int64})
    g = SimpleWeightedGraph(sv.n)
    for (u, v) in get_arcs(sv)
        penalized = get(x, (u, v), 0)
        cost = sv.c[u][v] + penalized * sv.d[u][v]
        add_edge!(g, u, v, cost)
    end
    sp = dijkstra_shortest_paths(g, 1)
    path_cost = sp.dists[sv.n]
    path = []
    next_node = sv.n
    while next_node != 0
        prepend!(path, next_node)
        next_node = sp.parents[next_node]
    end
    return path_cost, path
end

shortest_path_cost (generic function with 3 methods)

In [59]:
for (u, v) in get_arcs(sv)
    x[(u, v)] = 1
end

In [61]:
shortest_path_cost(sv, x)[2]

9-element Array{Any,1}:
  1
  2
  8
  9
 10
 15
 21
 25
 27

In [ ]:
rand()

In [ ]:
function random_branch(
        sv::Data,
        k::Int64,
        p::Int64,
        arcs_order::Array{Tuple{Int64, Int64}, 1},
        x::Dict{Tuple{Int64,Int64}, Int64}, 
    )
    
    